In [21]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import xarray as xr
import numpy as np
import pandas as pd

import ecephys_analyses as ea 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [23]:
xr.set_options(keep_attrs=True)

In [44]:
ea.NREM

['N1', 'N2']

In [19]:
subject = "Segundo"
experiment = 'sleep-homeostasis'
condition = 'light-period-circadian-match'
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
durations = np.arange(15, 50, 5)
subject_data = list()
for subject in subjects:
    hyp = ea.load_hypnogram(subject, experiment, condition)
    #spws = ea.load_spws(subject, experiment, condition)
    wake_hyp = hyp.keep_states(['Wake'])
    n_bouts = dict(subject=subject)
    for duration in durations:
        bouts = wake_hyp[wake_hyp.duration > pd.to_timedelta(duration, 'm')]
        n_bouts[duration] = len(bouts)
    subject_data.append(n_bouts)

df = pd.DataFrame.from_dict(subject_data)

In [20]:
df

,subject,15,20,25,30,35,40,45
0,Segundo,2,2,1,1,1,1,1
1,Valentino,3,2,2,1,0,0,0
2,Doppio,5,4,3,3,2,2,0
3,Alessandro,3,2,1,0,0,0,0
4,Allan,4,3,3,1,0,0,0
